# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

# TO DO (ctrl+f `!!!`):
- Add Day 3 slides (this cell)
- Check model update equation (Section 3)
- Explain difference between sensitivity kernel, misfit kernel, event kernel, gradient (section 3)



## Part 3: Seismic Imaging

- In this notebook we will bring all of the previous material together run a full seismic inversion to update a starting model. 
- We will generate a misfit kernel as shown in Day 2
- We use the misfit kernel to update a starting homogeneous halfspace model
- This new model is used to generate forward simulations, as shown in Day 1
- Waveforms from the initial model and updated model are compared to the True model
- Waveform misfit, introduced in Day 2, is used to determine if the model update improved waveform fit
- **Objective**: Illustrate model updates in the context of imaging. Relate manual tasks with nonlinear optimization algorithms and line searches
- These instructions should be run from inside the Docker container, using Jupyter Lab (see *Docker Preamble* in Day 0). 

-----------

**Relevant Links:** 
- Day 3 Slides !!! ADD THIS !!!
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_3a_imaging.ipynb
- Completed Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_3a_imaging.ipynb
- Day 0 Notebook (Container Testing): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Day 1A Notebook (Intro SPECFEM): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1a_intro_specfem2d.ipynb
- Day 1B Notebook (Fwd. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb
- Day 2A Notebook (Adj. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2a_kernels.ipynb

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.


## 1) Background

!!! TO DO !!!
Potential topics:
- Nonlinear optimization algorithms
- Line searches
- Model perturbations

## 2) Setting Up a SPECFEM2D Working Directory

- We want to set up a clean working directory to run SPECFEM2D inside. This will help us preserve our cloned repository and reduce file clutter.
- We will use SeisFlows to automate an inversion workflow **up to** kernel generation, to save time.
- SeisFlows will run a forward simulation, calculate misfit, and run an adjoint simulation.
- Experimental setup is: one source, one receiver, starting model is a homogeneous halfspace, target model is a perturbation checkerboard.
- **Objective**: Generate a misfit kernel which we can use to update our model manually. 
- **NOTE:** We will be doing all our work in the directory /home/scoped/work/day_3. All the following cells assume that we are in this directory, so you must evaluate the '%cd' command to ensure that cells work as expected.

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from IPython.display import Image
from scipy.integrate import simps
from seisflows.tools.specfem import Model, read_fortran_binary, write_fortran_binary

In [ ]:
# make sure we're in an empty working directory
! mkdir -p /home/scoped/work/day_3/sfexample_2
%cd /home/scoped/work/day_3/sfexample_2

# Run the example and stop after adjoint simulation
! seisflows examples setup 2 -r /home/scoped/specfem2d --event_id 1 --nsta 1 --niter 1 --with_mpi
! seisflows par stop_after evaluate_gradient_from_kernels
! seisflows submit

-----------

You will know that this workflow has completed successfully after the final log message   

```bash
2022-09-16 19:57:07 (I) | stop workflow at `stop_after`: evaluate_gradient_from_kernels
```

## 3) Manually Updating a Velocity Model

- We will use this gradient that was generated using SeisFlows to update our initial model. 
- With only have one source receiver pair, we can immediately check if the misfit of the waveforms has decreased by solving for $\chi$.
- In SeisFlows, we treat models and gradients as linear arrays, so model update can be memory-intensive, but is not CPU intensive.

!!! CHECK THIS EQUATION !!!

The Model update equation can be simple as: $m_{i+1} = m_{i} - \alpha \times g$ 
Where $m_{i}$ is the current model, $m_{i+1}$ is the updated model, $g$ is the gradient and $\alpha$ is a scale factor.

### Initial Model ($m_i$) and Target Model ($m_{true}$)

- The **initial/starting** model is our homogeneous halfspace model
- The homogeneous halfspace model defines a region with P-wave velocity 5.8km/s and S-wave velocity 3.5km/s. 
- This is the same model that we looked at in Day 1B and 2A 
- The **target/true** model is a checkerboard perturbation model
- Perturbations are roughly $\pm$10\% of the initial homogeneous halfspace Vp and Vs models
- The target model is used to generate 'data', which mimics real-world observations

In [ ]:
# Load in the initial model binary files, plot
m_init = Model("output/MODEL_INIT")
print(f"INITIAL MODEL\n{m_init.model}")
m_init.plot2d("vs")

In [ ]:
# Load the target model which has generated 'data'
m_true = Model("output/MODEL_TRUE")
print(f"TARGET MODEL\n{m_true.model}")
m_true.plot2d("vs")

### Misfit Kernel / Gradient ($g$)

!!! TO DO explain difference between sensitivity kernel, misfit kernel, event kernel, gradient !!!

- The gradient was generated by the interaction of our forward and adjoint wavefields.
- It represents the gradient of the misfit function and provides a search direction for model update.
- This gradient was generated using only one source-receiver pair
- This gradient was smoothed with a 2D Gaussian of vertical and horizontal half-widths of 5km
- This gradient is very similar to what we recovered after smoothing our misfit kernel in Day 2
- **NOTE**: Because the Gradient output directory does not contain coordinate information, we need to grab it from the model.


In [ ]:
# Load in Gradient binary files
gradient = Model("output/GRADIENT_01")

# Assign coordinate information which is used for plotting
gradient.coordinates = {}
gradient.coordinates["x"] = m_init.coordinates["x"]
gradient.coordinates["z"] = m_init.coordinates["z"]

print(f"GRADIENT\n{gradient.model}")
gradient.plot2d("vs_kernel")

### Updated Model ($m_{i+1}$)

- The updated model can be found by scaling the gradient and adding the negative gradient to the initial model. 
- Remember that the gradient represents maximum change, and since we are solving a minimization problem, we want to use the inverse gradient.
- The updated model is represented by equation: $m_{i+1} = m_{i} - \alpha \times g$ 
- Here we do **not** scale the gradient, so $\alpha$==1

In [ ]:
# We can use the SeisFlows Model class to update the model directly
m_update = m_init.copy()

# No scaling is applied here
m_update.update(vector=m_init.vector - gradient.vector)

print(m_update.model)

m_update.plot2d("vs")

#### Understanding the updated model
- Don't let the color scale fool you. Look at the values in the title to see how little the gradient changed the model
- We can see that because the gradient is **not** well scaled, the model updates only make slight changes to the Vs model ($\pm.0001$m/s give or take). 
- In order to make appreciable changes to the starting model, we need the **scale** the gradient. 
- There are many algorithms which provide scaling estimates for the gradient. 
- One thing we can try, is scaling by GTG$^{-1}$ (i.e,. the inverse of the dot product of the gradient itself)

In [ ]:
# Caclulating GTG^-1
gtg = np.dot(gradient.vector, gradient.vector)
gtg_inverse = gtg ** -1 

print(f"GTG^-1 = {gtg_inverse:.2f}")

In [ ]:
# Update the starting model with the scaled gradient
m_update = m_init.copy()
m_update.update(vector=m_init.vector - (gtg_inverse * gradient.vector))

print(m_update.model)

m_update.plot2d("vs")

#### Understanding the newly scaled model update
- The actual volumetric region added to the initial model is the same. We have only changed the amplitude
- The gradient is now more well scaled, and has updated our velocity model at most 25m/s. 
- We can try to use this updated velocity model to generate **new** synthetics
- We expect that the waveforms generated from this updated model will differ from our initial model synthetics
- We can compare waveform misfit for the new model synthetics against the true model to determine it's misfit value
- If misfit has reduced from the original model, then we're getting somewhere!

## 4) Forward Simulations w/ Updated Model

- We now have an updated model stored in the Python parameter `m_update`. 
- We'll use this updated model to generate a new set of synthetics
- As in Day 2, misfit will be calculated using a waveform misfit. 
- **Objective**: Determine if we have reduced waveform misfit through model update

### a) Setup SPECFEM2D working directory
- Let's start out by setting up a new SPECFEM2D working directory where we can manually run the SPECFEM binaries
- This task is very similar to Day 2 so we skip over detailed explanations

In [ ]:
# Setup block for our SPECFEM2D working directory
! mkdir /home/scoped/work/day_3/specfem2d_workdir
%cd /home/scoped/work/day_3/specfem2d_workdir

# Symlink the binary files located in bin/
! ln -s /home/scoped/specfem2d/bin .
# Copy over the DATA/ directory with Par_file, SOURCE and STATIONS
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
# Pick a pre-defined Par_file 
! cp DATA/Par_file_Tape2007_onerec DATA/Par_file
# Ensure we are using SOURCE #1
! cp DATA/SOURCE_001 DATA/SOURCE
# Ensure we are using only STATION #1
! head -1 DATA/STATIONS_checker > DATA/STATIONS
# Create the output directory
! mkdir OUTPUT_FILES

! ls

### Run SPECFEM2D to get model $m_i$ synthetics

- First we'll run SPECFEM2D to get model $m_i$ synthetics
- We'll later compare these to the updated model ($m_{i+1}$) synthetics
- We'll need to adjust some parameter file parameters before running

In [ ]:
# Tell SPECFEM to output binary model files
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.

# Run SPECFEM with the homogeneous halfspace model, 1 source, 1 station
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver.txt

In [ ]:
# A succesfully completed solver will write data files, expressed in the log
! tail -n 30 OUTPUT_FILES/output_solver.txt

### Setting the Updated Model as the 'Current' Model

- SPECEFM2D needs to be able to find the updated model we just created. 
- Luckily we can use SeisFlows to write these files in SPECFEM-readable formats.
- SPECFEM2D expects model files to be in the DATA/ directory. (**NOTE**: SPECFEM3D expects them to be in the `LOCAL_PATH` directory)
- We can OVERWRITE the updated parameters (in this case Vs and Vp) and leave the remainder of the model the same
- **IMPORTANT**: The `Par_file` parameter 'MODEL' must be set to `gll`, which tells SPECFEM to read the updated model files we have written

In [ ]:
# First we make sure that the original model files are stored somewhere safe
! mkdir MODEL_INIT
! cp -r DATA/*bin MODEL_INIT
! cp -r OUTPUT_FILES/*semd MODEL_INIT

print(f"parameters to be updated are: {m_update.parameters}")

# SeisFlows will overwrite the 'Vp' and 'Vs' binary files
m_update.write(path="DATA")

# Update the `Par_file` parameter 'MODEL' to tell SPECFEM to read model files from DATA/
! seisflows sempar -P DATA/Par_file model gll

### Run SPECFEM2D with updated model

- We can now run `xmeshfem2D` and `xspecfem2D` to retrieve synthetics for our updated model
- Since we set `MODEL`==`gll`, SPECFEM2D will know to read the **updated** model files we just wrote
- Because we are updating *model* files, we need to **rerun** `xmeshfem2D`
- This run will **overwrite** files stored in `OUTPUT_FILES/`. If you want to preserve the original files, you can copy the directory to a new path

In [ ]:
# Re-run the mesher and solver with the updated velocity model
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher_update.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver_update.txt

In [ ]:
# We can confirm that the updated model was used during the simulation by checking the solver log
! head -236 OUTPUT_FILES/output_solver_update.txt | tail -n 15

In [ ]:
# Make sure that the final model files are stored somewhere
! mkdir MODEL_UPDATE
! cp -r DATA/*.bin MODEL_UPDATE
! cp -r OUTPUT_FILES/*semd MODEL_UPDATE
! ls MODEL_UPDATE

## 5) Comparing Synthetics for Initial and Updated Models

- We now have 2 synthetic seismograms. 1 for the initial model, 1 for the updated model
- In a real inversion, we would have 2$\times$N seismograms, corresonding to N stations 
- If we plot the two waveforms together we should see that they differ
- Our aim is to check both synthetics against the True model synthetics (checkerboard model)
- **Objective**: Determine if our updated model has reduced waveform misfit

### Compare Initial and Update synthetics

- We first want to see if the model update changed the synthetic waveforms
- We'll re-use some code snippets from Day 2 to plot synthetics
- Initial model synthetics: *red*
- Updated model synthetics: *purple*
- Initial - Update (waveform difference): *blue*

In [ ]:
t_init, d_init = np.loadtxt("MODEL_INIT/AA.S000000.BXY.semd").T
t_update, d_update = np.loadtxt("MODEL_UPDATE/AA.S000000.BXY.semd").T

# Plot both waveforms using Matplotlib
plt.plot(t_init, d_init, c="r", label="MODEL INIT; SYNTHETIC")
plt.plot(t_update, d_update, c="magenta", label="MODEL UPDATE; 'SYNTHETIC'")
plt.plot(t_init, d_init - d_update, c="b", label="SYNTHETIC DIFFERENCE")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.legend()
plt.show()

If we zoom into the time window where the two waveforms are different, we can better identify how the model update has affected the synthetic waveforms.
The time window of waveform difference (min/max amplitude of the blue wiggle) spans roughly *T0=60s* to *T1=85s*

In [ ]:
# Create the same plot as above, but zoom in on time window of waveform difference
plt.plot(t_init, d_init, c="r", label="MODEL INIT; SYNTHETIC")
plt.plot(t_update, d_update, c="magenta", label="MODEL UPDATE; 'SYNTHETIC'")
plt.plot(t_init, d_init - d_update, c="b", label="SYNTHETIC DIFFERENCE")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.xlim([60, 85])
plt.legend()
plt.show()

In [ ]:
# Plot min/max values to see if model update has affected waveform amplitude
print(f"INITIAL MODEL WAVEFORM: \n   MIN={d_init.min():.3f}; MAX={d_init.max():.3f}")
print(f"TARGET MODEL WAVEFORM:  \n   MIN={d_true.min():.3f}; MAX={d_true.max():.3f}")


#### Understanding waveform differences
- Visually the waveforms look almost the same. 
- Zooming in on the time window of waveform difference shows slight waveform differences
- The updated synthetics (purple) are slightly *delayed* with respect to the initial model synthetics (red)
- Updated synthetics also have slightly larger minimum and maximum amplitudes
- Larger model update perturbation (scaling the gradient) would likely have increased the waveform differences seen here

### Generating True/Target model synthetics

- The final check here is to calculate waveform misfit for the initial and updated models, with respect to the Target checkerboard model.
- We will first need to generate checkerboard synthetics, a task that we performed in Day 2
- We will also need to re-define our waveform misfit function
- **Objective**: Generate True model 'data' and compare against the two synthetic waveforms we have

In [ ]:
# This new par file OVERWRITES the changes we made previously 
! cp -f DATA/Par_file_Tape2007_132rec_checker DATA/Par_file

! seisflows sempar -P DATA/Par_file NSTEP 5000  # 4800 -> 5000 to match the other Par_file
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1

# Ensure that SPECFEM can find the checkerboard model by naming it correctly
! cp -f DATA/model_velocity.dat_checker DATA/proc000000_model_velocity.dat_input

# Re-run mesher and solver for the Target model
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher_true.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver_true.txt

In [ ]:
# Store True model binary files and synthetics
! mkdir MODEL_TRUE
! cp -r DATA/*.bin MODEL_TRUE
! cp -r OUTPUT_FILES/*semd MODEL_TRUE
! ls MODEL_TRUE

### Compare Initial, Update synthetics with Target data

- We now plot the target model data on top of the waveforms we showed earlier
- Target model synthetics (i.e., 'data') was generated using the checkerboard model
- Initial model synthetics: *red*
- Updated model synthetics: *purple*
- Initial - Update (waveform difference): *blue*
- Target model synthetics: *black*

In [ ]:
# Plot waveforms with the True model, which will be colored black
t_true, d_true = np.loadtxt("MODEL_TRUE/AA.S000000.BXY.semd").T

plt.plot(t_init, d_init, c="r", label="MODEL INIT; SYNTHETIC")
plt.plot(t_true, d_true, c="k", label="MODEL TRUE; 'DATA'")
plt.plot(t_update, d_update, c="magenta", label="MODEL UPDATE; 'SYNTHETIC'")
plt.plot(t_init, d_init - d_update, c="b", label="SYNTHETIC DIFFERENCE")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.legend()
plt.show()

In [ ]:
# Plotting a zoom on the waveform difference time window
plt.plot(t_init, d_init, c="r", label="MODEL INIT; SYNTHETIC")
plt.plot(t_true, d_true, c="k", label="MODEL TRUE; 'DATA'")
plt.plot(t_update, d_update, c="magenta", label="MODEL UPDATE; 'SYNTHETIC'")
plt.plot(t_init, d_init - d_update, c="b", label="SYNTHETIC DIFFERENCE")
plt.xlim([60, 85])
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.legend()
plt.show()

#### Understanding waveform differences
- Target model synthetics (black) are *delayed* compared to initial model synthetics (red), by approximately 3s
- Target model synthetics (black) show *larger* amplitudes compared to initial model synthetics (red)
- By visual inspection, the updated model synthetics (purple) appear to be moving *towards* the target synthetics (black)
- Visual inspection is a qualitative measure, we would rather **quantify** waveform misfit to determine if the updated synthetics better fit target synthetics

### Quantfiying Waveform Misfit

- As in Day 2 we will calculate a waveform misfit $\chi$, whose equation is: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$
- In the equation above, $d(t)$ represents 'data' and $s(t)$ represents 'synthetics'
- We want to compare data with both the True model and the updated model, so we need to calculate $\chi$ twice
- **Objective**: Determine if the model update has reduced waveform misfit

In [ ]:
# Integrate using scipy
dt = t_true[1] - t_true[0]  # represents the time step, or `dt`. Same for all waveforms

# Use simpsons rule for integration
misfit_init = 1/2 * simps((d_true - d_init)**2, dx=dt)
misfit_update = 1/2 * simps((d_true - d_update)**2, dx=dt)

print(f"misfit_init = {misfit_init:.3E}")
print(f"misfit_true = {misfit_update:.3E}")
print(f"init - true = {misfit_init - misfit_update:.3E}")

---------------

**Mission accomplished!**
- We have successfully reduced the waveform misfit for a single source receiver pair by updating our velocity model.
- The misfit reduction is relatively small; it is tied to the gradient and chosen scale factor
- We could increase the scale factor to achieve a more favorable misfit reduction, however if we go too far, we may end up increasing the misfit.
- Algorithmically, this step is defined in a `line search`, which attempts to find an appropriate scale factor for model update
- There are many types of nonlinear optimization algorithms (e.g., gradient descent, L-BFGS) which are used to determine scale factors in an effort to minimize misfit
- **NOTE** each evaluation of a line search requires **re-running** the forward simulation with a trial model, it is therefore ideal to find the best performing algorithm to keep computational costs down 
- See e.g., [Modrak and Tromp (2016)](https://academic.oup.com/gji/article/206/3/1864/2583505) for exploration of how various algorithms perform in the context of seismic inverison


## 6) A Multi-event, Multi-station Inversion

- Real seismic inversions use multiple events recorded at multiple stations to increase coverage and reduce nonuniqueness
- Here we attempt a *multi-event* ($N=10$), multi-station ($S=10$) model update with the tools we just covered
- We forgo visualizations and detailed explanations for the sake of brevity
- The inversion workflow is the same as above: 
    0) **data**: run $N$ forward simulations to generate *data* using target model ($m_{true}$) for $N$ events and $S$ stations
    1) **synthetics**: run $N$ forward simulations to generate synthetics through starting model ($m_{init}$) for $N$ events and $S$ stations
    2) **misfit**: quantify data-synthetic misfit ($\chi_{init}$) and generate adjoint sources for each source-receiver pair ($N\times S$ waveforms)
    3) **kernels**: generate misfit kernels with $N$ adjoint simulation
    4) **update**: scale gradient and update initial model to get trial model: $m_{try}$
    5) **line search**: run $N$ forward simulations using trial model $m_{try}$ to get $N \times S$ new synthetic waveforms
    6) **line search**: calculate misfit ($\chi_{try}$) and determine if misfit is reduced ($\chi_{try} < \chi_{init}$?)
    7) **line search**: if misfit is not reduced ($\chi_{try} >=  \chi_{init}$), repeat steps 5-7 with newly scaled gradient
    8) **line search**: if misfit is reduced, accept trial model as newly updated model ($m_{try} \rightarrow m_{01}$)
    9) **finalize**: set newly updated model as initial model ($m_{01} \rightarrow m_{init}$) and repeat steps 1-9
    
    
Much of this experimental setup was shown in Day 2 but in Section (a) we will plot the sources and stations to be used here to provide some context to the inversion problem that will be run.

### a) Generating 100 Target Model Synthetics

- Checkerboard model used to generate 'data'
- Homogeneous halfspace model as our 'initial' model to generate synthetics
- 10 sources and 10 stations with pre-defined locations

In [ ]:
# make sure we're in an empty working directory
! mkdir -p /home/scoped/work/day_3/inversion_example
%cd /home/scoped/work/day_3/inversion_example

# Setup SPECFEM2D working directory
! ln -s /home/scoped/specfem2d/bin .
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
! cp -f DATA/Par_file_Tape2007_132rec_checker DATA/Par_file
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.
! head -10 DATA/STATIONS_checker > DATA/STATIONS  # 10 stations
! mkdir OUTPUT_FILES

# This new par file OVERWRITES the changes we made previously 
! seisflows sempar -P DATA/Par_file NSTEP 5000  # 4800 -> 5000 to match the other Par_file
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1

# Ensure that SPECFEM can find the checkerboard model by naming it correctly
! cp -f DATA/model_velocity.dat_checker DATA/proc000000_model_velocity.dat_input

In [ ]:
# Small code block to plot source-receiver geometry with text labels
# Grab station locations
sta_x, sta_z = np.genfromtxt("DATA/STATIONS", dtype=float, usecols=[2, 3]).T
# Grab event locations
ev_x, ev_z = [], []
for i in range(1, 11):
    source_file = f"DATA/SOURCE_{i:0>3}"
    with open(source_file, "r") as f:
        lines = f.readlines()
    # Trying to break apart the following line
    # 'xs = 299367.72      # source location x in meters\n'
    xs = float(lines[2].split("=")[1].split("#")[0].strip())
    zs = float(lines[3].split("=")[1].split("#")[0].strip())
    
    ev_x.append(xs)
    ev_z.append(zs)
# Plot SOURCES and STATIONS geometry
for i, (x, z) in enumerate(zip(ev_x, ev_z)):
    plt.scatter(x, z, c="y", marker="*", edgecolor="k", s=50)
    plt.text(x, z, f"S{i+1:0>2}", fontsize=9, c="r")  # SOURCE numbering starts at 1
for i, (x, z) in enumerate(zip(sta_x, sta_z)):
    plt.scatter(x, z, c="c", marker="v", s=40, edgecolor="k")
    plt.text(x, z, f"R{i:0>2}", fontsize=9)
# Finalize plot labels
plt.xlabel("X [m]")
plt.ylabel("Z [m]")
plt.xlim([0, 480E3])
plt.ylim([0, 480E3])
plt.title("SOURCE-RECEIVER GEOMETRY")

In [ ]:
# Define a re-usable function because we will be running a lot of forward simulations
def run_fwd_mesher_solver(ntask, save_synthetics, save_forward=False):
    """
    Run the mesher and solver `ntask` times. Save the resulting synthetics
    to path `save_synthetics`
    
    :type ntask: int
    :param ntask: number of sources to run
    :type save_synthetics: str
    :param save_synthetics: relative directory name to save synthetics to
    """
    print(f"Generating synthetics for {ntask} sources")
    # Run 10 forward simulations using Target model, generating 'data' for 10 stations. 
    for i in range(1, ntask + 1):
        # Choose the task/source to run mesher/sovler with
        source_fid = f"SOURCE_{i:0>3}"
        print(f"Running mesher and solver for {source_fid}")
        source_path = f"DATA/{source_fid}"

        # Set the correct source for SPECFEM to find
        if os.path.exists("DATA/SOURCE"):
            os.remove("DATA/SOURCE")
        shutil.copy(source_path, "DATA/SOURCE")

        # Run xmeshfem and xspecfem. 
        # Note: We actually only have to run the mesher once to set up the database files, 
        #     but it's cheap so we do it for each source. In 3D you may want to skip that step.
        ! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher.txt
        ! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver.txt

        # Move the just-generated synthetic waveforms so they don't get overwritten
        os.makedirs(os.path.join(source_fid, save_synthetics))
        for src in glob("OUTPUT_FILES/*.semd"):
            dst = os.path.join(source_fid, save_synthetics, os.path.basename(src))
            os.rename(src, dst)
        
        # Save forward arrays stored on disk if available
        if save_forward:
            print(f"saving forward arrays {source_fid}")
            save_fwd_dir = os.path.join(source_fid, "SAVE_FORWARD")
            if not os.path.exists(save_fwd_dir):
                os.makedirs(save_fwd_dir)
            # Save fwd arrays are the last frame and boundary conditions
            fid_templates = ["lastframe_elastic*.bin", "absorb_elastic_*.bin"]
            for fid in fid_templates:
                for src in glob(f"OUTPUT_FILES/{fid}"):
                    dst = os.path.join(save_fwd_dir, os.path.basename(src))
                    os.rename(src, dst)
        
    print("Finished")

In [ ]:
# Run the mesher and solver for the Initial model and 10 sources.
run_fwd_mesher_solver(ntask=10, save_synthetics="TARGET_DATA", save_forward=False)

### b) Generating 100 Initial Model Synthetics

- Now we generate synthetics through out homogeneous halfspace initial model
- We will **also** need the forward wavefield (output from `SAVE_FORWARD`) from the forward simulation of each $N$ events.
- The saved forward wavefield is a snapshot of the forward wavefield at the last time step, used to back-reconstruct the forward wavefield
- The saved forward wavefield is required to run each of the $N$ subsquent adjoint simulations
- For large domains (and especially in 3D), saved forward wavefields are **large** files (on the order of GB)
- It is *not* preferable to save these large files permanently. Ideally we would calculate misfit, run adjoint simulations, and then remove the forward wavefield
- We will follow that practice here by deleting the forward wavefield after each adjoint simulation

In [ ]:
%cd /home/scoped/work/day_3/inversion_example

# Re-set the SPECFEM2D Par_file which is set up for the Initial model
! cp -f DATA/Par_file_Tape2007_onerec DATA/Par_file

# Edit Par_file to get the correct output files for Initial model
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.
! seisflows sempar -P DATA/Par_file save_forward .true.

In [ ]:
# Run mesher and solver for the Target model and 10 sources. Save forward arrays for subsequent adjoint simulations
run_fwd_mesher_solver(ntask=10, save_synthetics="SYNTHETICS", save_forward=True)

### c) Quantifying misfit and generating adjoint sources

- Now  we need to quantify misfit for all $N \times S$ waveforms
- We also need to generate $N \times S$ adjoint sources for subsequent adjoint simulations
- We'll use a waveform difference misfit for simplicity: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$
- The corresponding waveform difference adjoint source: $f^\dagger (t) = s(t) - d(t)$

In [ ]:
# First we define a function that will calculate misfit and generate adjoint sources
def quantify_misfit(data_fid, syn_fid): 
    """
    Quantify misfit and generate adjoint source
    
    :type data_fid: str
    :param data_fid: filename and path to data array
    :type syn_fid: str
    :param syn_fid: filename and path to synthetic array
    """
    t_data, d_data = np.loadtxt(data_fid).T
    t_syn, d_syn = np.loadtxt(syn_fid).T
    dt = t_syn[1] - t_syn[0]  # dt represents the time step
    
    # Calculate waveform difference misfit
    misfit = 1/2 * simps((d_data - d_syn)**2, dx=dt)
    
    # Create the time-amplitude array for adjoint source
    d_adj = d_init - d_true
    adj_src = np.vstack((t_syn, d_adj)).T
    
    return misfit, adj_src    

In [ ]:
# In this code block we quantify misfit and store results accordingly
for src in sorted(glob("SOURCE_???")):
    print(f"Quantifying misfit for {src}")
    
    # Set up directories to store misfit and adjoint sources
    adj_src_dir = os.path.join(src, "ADJOINT_SOURCE")
    misfit_file = os.path.join(src, "MISFIT.txt")
    
    if not os.path.exists(adj_src_dir):
        os.makedirs(os.path.join(src, "ADJOINT_SOURCE"))
        
    if os.path.exists(misfit_file):
        os.remove(misfit_file)

    # Loop through each of the syntheitc waveforms
    for syn_fid in sorted(glob(os.path.join(src, "SYNTHETICS", "*"))):
        # e.g., AA.S000001.BXY.semd
        fid_base = os.path.basename(syn_fid)
        
        print(f"\t{fid_base}", end="; ")
        data_fid = os.path.join(src, "TARGET_DATA", fid_base)
        
        # Calculate the misfit and generate adjoint source    
        misfit, adj_src = quantify_misfit(data_fid, syn_fid)
        
        # Write misfit to file
        print(f"MISFIT={misfit:.3E}")
        with open(misfit_file, "a") as f:
            f.write(f"{misfit}\n")
        
        # Write adjoint source to file
        # Note that SPECFEM expects adjoint sources to be suffixed '.adj.
        np.savetxt(os.path.join(adj_src_dir, fid_base.replace("semd", "adj")), adj_src)

In [ ]:
# We can take a look at the files we just created
! ls SOURCE_001
! echo 
! echo "> ADJOINT SOURCES"
! ls SOURCE_001/ADJOINT_SOURCE
! echo 
! echo "> SOURCE_001/ADJOINT_SOURCE/AA.S000000.BXY.adj"
! head SOURCE_001/ADJOINT_SOURCE/AA.S000000.BXY.adj
! echo
! echo "> SOURCE_001/MISFIT.txt"
! head SOURCE_001/MISFIT.txt

In [ ]:
# Plot an example of data, synthetic and adjoint source
t_init, d_init = np.loadtxt("SOURCE_001/SYNTHETICS/AA.S000000.BXY.semd").T
t_true, d_true = np.loadtxt("SOURCE_001/TARGET_DATA/AA.S000000.BXY.semd").T
t_adjs, d_adjs = np.loadtxt("SOURCE_001/ADJOINT_SOURCE/AA.S000000.BXY.adj").T

plt.plot(t_init, d_init, c="r", label="SYNTHETIC")
plt.plot(t_true, d_true, c="k", label="'DATA'")
plt.plot(t_adjs, d_adjs, c="g", label="ADJOINT SOURCE")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")
plt.title("DATA-SYNTHETIC COMPARISON\nSOURCE 001; AA.S000000.BXY")
plt.xlim([25, 150])
plt.legend()
plt.show()

#### Understanding misfit quantification
- For each of our $N$ sources we have quantified misfit for $S$ stations
- Each synthetic has a corresponding adjoint source, which is stored in `SOURCE_???/ADJOINT_SOURCE`
- Each adjoint source is equal to the waveform difference between data and synthetic
- Each adjoint source has a corresponding misfit value
- Misfit values are stored in a text file which can be summed to calculate the overall **event misfit**
- The adjoint sources will be used to run $N$ adjoint simulations
- The misfit value will be used to determine if misfit is reduced during the line search

### d) Running $N$ adjoint simulations to generate event kernels

- Now we need to run the adjoint simulations to generate event kernels
- Event kernels tell us what parts of the model are sensitive to the misfit function for **EACH** event
- Adjoint simulations take $S$ adjoint sources as input 
- **NOTE**: We need to place the adjoint sources in the correct location and also tell SPECFEM that we are running an adjoint simulation

In [ ]:
def run_adj_solver(ntask):
    """
    Counterpart to `run_fwd_mesher_solver`, runs adjoint simulations for `ntask` events
    and saves the resulting kernels
    """
    for i in range(1, ntask + 1):
        # Choose the task/source to run mesher/sovler with
        source_fid = f"SOURCE_{i:0>3}"
        assert(os.path.exists(source_fid)), f"no corresponding fwd results for {source_fid}"
        
        print(f"Running adjoint simulation for {source_fid}")
        source_path = f"DATA/{source_fid}"

        # Set the correct source for SPECFEM to find
        if os.path.exists("DATA/SOURCE"):
            os.remove("DATA/SOURCE")
        shutil.copy(source_path, "DATA/SOURCE")
                
        # Symlink adjoint sources so SPECFEM can find them
        if os.path.exists("SEM"):
            os.remove("SEM")
        os.symlink(os.path.join(source_fid, "ADJOINT_SOURCE"), "SEM")
        
        # Copy the saved forward array so that SPECFEM can use it
        for src in glob(os.path.join(source_fid, "SAVE_FORWARD", "*")):
            dst = os.path.join("OUTPUT_FILES", os.path.basename(src))
            if os.path.exists(dst):
                os.remove(dst)
            shutil.copy(src, dst)
            
        # Run the adjoint simulation
        ! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/adjoint_solver.txt
        
        # # Move the resulting kernel outputs to disk
        kernel_dir = os.path.join(source_fid, "KERNELS")
        os.makedirs(kernel_dir)
        for src in glob("OUTPUT_FILES/*kernel.bin"):
            dst = os.path.join(kernel_dir, os.path.basename(src))
            os.rename(src, dst)
            
        print("finished")

In [ ]:
# Set up the Par_file to run an adjoint simulation and output kernels in expected output
! seisflows sempar -P DATA/Par_file simulation_type 3
! seisflows sempar -P DATA/Par_file save_ASCII_kernels .false.

# Run adjoint simluation for each N sources
run_adj_solver(ntask=10)

### e) Summing kernels and scaling gradient

- Now we have $N$=10 event kernels, each representing misfit sensitivity for each of the $N$ events
- We need to sum the event kernels into a misfit kernel, which represents misfit sensitivity for **all** $N$ events
- We can use the `xcombine_sem` executable to sum these kernels
- Before perturbing the initial model, we need to *scale* the misfit kernel 
- The scaled misfit kernel is the gradient, which we can use to perturb the initial model
- The call structure for `xcombine_sem` is:  *mpirun -n NPROC bin/xcombine_sem KERNEL_NAMES INPUT_FILE OUTPUT_DIR*

In [ ]:
# In this block we sum event kernels into a misfit kernel
# First we need to tell SPECFEM where all our kernel files are
with open("kernel_paths.txt", "w") as f:
    for fid in sorted(glob("SOURCE_???/KERNELS")):
        f.write(f"{os.path.abspath(fid)}\n")

# Make a directory to store these files
! mkdir GRADIENT

# Then we run `xcombine_sem` to combine kernels for various kernel names
! mpirun -n 1 bin/xcombine_sem alpha_kernel,beta_kernel kernel_paths.txt GRADIENT 

# Rename 'alpha' to Vp and 'beta' to Vs for easier name recognition
for src in glob("GRADIENT/*"):
    if "alpha" in src:
        os.rename(src, src.replace("alpha", "vp"))
    elif "beta" in src:
        os.rename(src, src.replace("beta", "vs"))

In [ ]:
# In this cell we'll scale the gradient (Vp and Vs only) by GTG^-1
vp_kernel = read_fortran_binary("GRADIENT/proc000000_vp_kernel.bin")
vs_kernel = read_fortran_binary("GRADIENT/proc000000_vs_kernel.bin")

# The gradient starts out as a vector of kernels for all parameters
gradient = np.hstack((vp_kernel, vs_kernel))

# Caclulating GTG^-1
gtg = np.dot(gradient, gradient)
gtg_inverse = gtg ** -1 

print(f"GTG^-1 = {gtg_inverse:.2f}")

# Scaling the gradient by GTG^-1
gradient = gradient * gtg_inverse

In [ ]:
# Plotting the gradient
x = read_fortran_binary("DATA/proc000000_x.bin")
z = read_fortran_binary("DATA/proc000000_z.bin")
n = len(gradient) // 2
g_vp = gradient[:n]
g_vs = gradient[n:]

plt.tricontourf(x, z, g_vs, levels=125, cmap="RdYlGn")
plt.colorbar()

# Plot SOURCES and STATIONS geometry
for i, (x, z) in enumerate(zip(ev_x, ev_z)):
    plt.scatter(x, z, c="y", marker="*", edgecolor="k", s=50)
    plt.text(x, z, f"S{i+1:0>2}", fontsize=9, c="r")  # SOURCE numbering starts at 1
for i, (x, z) in enumerate(zip(sta_x, sta_z)):
    plt.scatter(x, z, c="c", marker="v", s=40, edgecolor="k")
    plt.text(x, z, f"R{i:0>2}", fontsize=9)

plt.xlabel("X [m]")
plt.ylabel("Z [m]")
plt.title("Gradient Vs")

!!! DISCUSS THIS FIGURE !!!

### f) Smoothing the gradient

- The gradient above shows a lot of small scale heterogeneity
- This may be due to the summation of $N$=10 event kernels
- We can use regularization to smooth out the gradient to get a more conservative picture of model update
- We use the `xsmooth_sem` executable to smooth our kernels

 The usage of `xsmooth_sem` is given as
 ```bash
 USAGE:  mpirun -np NPROC bin/xsmooth_sem SIGMA_H SIGMA_V KERNEL_NAME INPUT_DIR OUPUT_DIR GPU_MODE
 ```
 We will need to choose values and directories to make this work
  - `SIGMA_H`: Horizontal smoothing length [m] representing the horizontal half-width of the Gaussian  
  - `SIGMA_Z`: Vertical smoothing length [m] representing the vertical half-width of the Gaussian  
  - `KERNEL_NAME`: The name of the kernel we want to smooth. Must match filename, so `proc000000_vs_kernel.bin` will correspond to kernel name `vs_kernel`  
  - `INPUT_DIR`: where SPECFEM can find the kernel files
  - `OUTPUT_DIR`: Where SPECFEM should output the SMOOTHED kernels 
  - `GPU_MODE`: Use GPU acceleration to speed up the smoothing operation (.true. or .false.)

In [ ]:
# SPECFEM Requires that some of the model files be discoverable alongside the kernels
! cp -f GRADIENT/*.bin DATA/
! mpirun -n 1 bin/xsmooth_sem 25000 25000 vs_kernel DATA/ GRADIENT/ .false.
! mpirun -n 1 bin/xsmooth_sem 25000 25000 vp_kernel DATA/ GRADIENT/ .false.

In [ ]:
# Plotting the smoothed gradient
x = read_fortran_binary("DATA/proc000000_x.bin")
z = read_fortran_binary("DATA/proc000000_z.bin")
vp_kernel = read_fortran_binary("GRADIENT/proc000000_vp_kernel_smooth.bin")
vs_kernel = read_fortran_binary("GRADIENT/proc000000_vs_kernel_smooth.bin")

# The gradient starts out as a vector of kernels for all parameters
gradient = np.hstack((vp_kernel, vs_kernel))

# Caclulating GTG^-1
gtg = np.dot(gradient, gradient)
gtg_inverse = gtg ** -1 

print(f"GTG^-1 = {gtg_inverse:.2f}")

# Scaling the gradient by GTG^-1
gradient = gradient * gtg_inverse

n = len(gradient) // 2
g_vp = gradient[:n]
g_vs = gradient[n:]

plt.tricontourf(x, z, g_vs, levels=125, cmap="RdYlGn")
plt.colorbar()

# Plot SOURCES and STATIONS geometry
for i, (x, z) in enumerate(zip(ev_x, ev_z)):
    plt.scatter(x, z, c="y", marker="*", edgecolor="k", s=50)
    plt.text(x, z, f"S{i+1:0>2}", fontsize=9, c="r")  # SOURCE numbering starts at 1
for i, (x, z) in enumerate(zip(sta_x, sta_z)):
    plt.scatter(x, z, c="c", marker="v", s=40, edgecolor="k")
    plt.text(x, z, f"R{i:0>2}", fontsize=9)
    
# Connect sources and receivers with lines
for (ex, ez) in zip(ev_x, ev_z):
    for (sx, sz) in zip(sta_x, sta_z):
        plt.plot((ex, sx), (ez, sz), c="k", lw=.25)


plt.xlabel("X [m]")
plt.ylabel("Z [m]")
plt.title("Gradient Vs")

### g) Perturbing the initial model

- We can perturb the initial model with the gradient to generate an update model ($m_{try}$)
- We'll use this updated model to run forward simulations
- Quantifying misfit of these new synthetics, we can determine if our misfit is reduced
- Successful reduction of misfit equals a successful line search

In [ ]:
# !!! MOVE THIS TO 
# First we store the starting model incase we need it again
! mkdir MODEL_INIT
! mkdir MODEL_01

# These files define a SPECFEM2D model
for tag in ["x", "z", "rho", "vp", "vs", "NSPEC_ibool", "jacobian"]:
    src = f"DATA/proc000000_{tag}.bin"
    # Copy to initial model directory
    dst = f"MODEL_INIT/{os.path.basename(src)}"
    shutil.copy(src, dst)
    # Copy to Model 01 directory, we will overload some of these
    dst = f"MODEL_01/{os.path.basename(src)}"
    shutil.copy(src, dst)
    
# Now we load the initial model velocity so we can overload them
vp = read_fortran_binary("MODEL_01/proc000000_vp.bin")
vs = read_fortran_binary("MODEL_01/proc000000_vs.bin")

# Perturb the initial model by the gradient
vp += g_vp
vs += g_vs

# Over write Vp and Vs for Model 01 with perturbed model
write_fortran_binary(vp, "MODEL_01/proc000000_vp.bin")
write_fortran_binary(vs, "MODEL_01/proc000000_vs.bin")

### g) Run forward simulations with M01, quantify misfit

- Now that we have a perturbed model (Model 01; M01) we can use it to run forward simulations
- We can calculate misfit for each of the new synthetics
- We can compare the overall misfit of the initial model to Model 01
- **Objective**: determine if we have reduced waveform misfit through our model perturbation

In [ ]:
# Set the correct velocity model where SPECFEM expects it
for tag in ["x", "z", "rho", "vp", "vs", "NSPEC_ibool", "jacobian"]:
    src = f"MODEL_01/proc000000_{tag}.bin"
    dst = f"DATA/proc000000_{tag}.bin"
    if os.path.exists(dst):
        os.remove(dst)
    shutil.copy(src, dst)

# Reset the simulation type to forward
! seisflows sempar -P DATA/Par_file simulation_type 1
! seisflows sempar -P DATA/Par_file model gll

run_fwd_mesher_solver(ntask=10, save_synthetics="SYNTHETICS_01", save_forward=False)

In [ ]:
# In this code block we quantify misfit and store results accordingly
for src in sorted(glob("SOURCE_???")):
    print(f"Quantifying misfit for {src}")
    
    # Set up directories to store misfit and adjoint sources
    misfit_file = os.path.join(src, "MISFIT_01.txt")

    if os.path.exists(misfit_file):
        os.remove(misfit_file)

    # Loop through each of the syntheitc waveforms
    for syn_fid in sorted(glob(os.path.join(src, "SYNTHETICS_01", "*"))):
        # e.g., AA.S000001.BXY.semd
        fid_base = os.path.basename(syn_fid)
        
        print(f"\t{fid_base}", end="; ")
        data_fid = os.path.join(src, "TARGET_DATA", fid_base)
        
        # Calculate the misfit and generate adjoint source    
        misfit, adj_src = quantify_misfit(data_fid, syn_fid)
        
        # Write misfit to file
        print(f"MISFIT={misfit:.3E}")
        with open(misfit_file, "a") as f:
            f.write(f"{misfit}\n")

h) Comparing misfit between Initial and Updated models
- Now we have two files containing misfit, one for the initial model, and one for the updated model (M01)
- Each line in the misfit file corresponds to the overall misfit for a single event
- We can compare them by taking the average of all lines in the misfit file

!!! Put the misfit equation here !!!

In [ ]:
# Calculate misfit from each source
misfit_init, misfit_m01 = [], []

for src in sorted(glob("SOURCE_???")):
    m_init = np.loadtxt(os.path.join(src, "MISFIT.txt"))
    m_01 = np.loadtxt(os.path.join(src, "MISFIT_01.txt"))
    
    misfit_init.append(m_init.sum() / len(m_init))
    misfit_m01.append(m_01.sum() / len(m_01))
    
print(f"MISFIT INITIAL MODEL = {sum(misfit_init) / len(misfit_init)}")
print(f"MISFIT UPDATED MODEL = {sum(misfit_m01) / len(misfit_m01)}")

## 7) Automating Inversions using SeisFlows

- Of course, now that we have gone through the rigorous manual exercise of updating a velocity model, we can see how SeisFlows automates this procedure.
- SeisFlows contains a built-in optimization library, which features gradient-descent, non-linear conjugate gradient (NLCG), and L-BFGS nonlinear optimization alogirthms
- This library takes care of the scaling of the gradient, automating the line search and model update
- Under the hood, SeisFlows essentialy performs what we just did, scaling the velocity model, re-running forward simulations and comparing misfit values
- Misfit is either calculated with the default preprocessing module, or using Pyatoa, a software designed specifically for waveform misfit quantification.

In [ ]:
! mkdir /home/scoped/work/day_3/sfexample_2
%cd /home/scoped/work/day_3/sfexample_2
! seisflows examples setup 2 -r /home/scoped/specfem2d --event_id 1 --niter 1 --nsta 1 --with_mpi

!!! analyze log message above here !!!

In [ ]:
! seisflows submit

-------------

The task will be finished when you see the log message:

```bash
////////////////////////////////////////////////////////////////////////////////
                             COMPLETE ITERATION 01                              
////////////////////////////////////////////////////////////////////////////////
2022-09-16 21:25:38 (I) | setting current iteration to: 2
```

In [ ]:
! echo "> output/ contains models, gradients, etc."
! ls output
! echo

! echo "> MODEL directories contain SPECFEM formatted model files"
! ls output/MODEL_INIT
! echo

! echo "> GRADIENT directories contain regularized misfit kernels"
! ls output/GRADIENT_01
! echo

In [ ]:
# Models/gradients can be plotted directly from the command line
! seisflows plot2d MODEL_01 vs --savefig m_01_vs.png
Image("m_01_vs.png")

Under the hood, SeisFlows is simply running a number of individual SPECFEM working directories

In [ ]:
! ls scratch/solver/001